<a href="https://colab.research.google.com/github/buthaina279/LSTM_MIMIC-III/blob/main/join_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Extract cardiovascular patients from the raw MIMIC-III

In [ ]:
%%bigquery --project mimic-351822 cv_patients
SELECT
    d.subject_id, i.icustay_id, d.hadm_id
    , CASE when  d.icd9_code between '390%' and '460%' THEN 1 ELSE 0 END AS cv_icu
FROM physionet-data.mimiciii_clinical.diagnoses_icd d 
    INNER JOIN physionet-data.mimiciii_clinical.icustays i
    ON i.hadm_id = d.hadm_id AND i.subject_id = d.subject_id
    
WHERE seq_num IS NOT NULL
and d.icd9_code between '390%' and '460%'
GROUP BY i.icustay_id, d.subject_id, d.hadm_id, cv_icu
order by subject_id

In [ ]:
#for google drive
DATA_FILEPATH = "/content/drive/MyDrive/final_project/"

In [ ]:
import pandas as pd, numpy as np
df_vitals_labs = pd.read_csv(DATA_FILEPATH + 'vitals_labs.csv',index_col=0) 

In [ ]:
df_stat = pd.read_csv(DATA_FILEPATH + 'statistics.csv',index_col=0) 

##cardiovascular patients 

In [ ]:
cv_patients.head()

,subject_id,icustay_id,hadm_id,cv_icu
0,3,211552,145834,1
1,6,228232,107064,1
2,9,220597,150750,1
3,12,232669,112213,1
4,13,263738,143045,1


In [ ]:
cv_patients.shape

(45155, 4)

In [ ]:
cv_patients['cv_icu'].unique()

array([1])

Since cv_icu has only one unique value, it will be removed



In [ ]:
cv_patients.drop(['cv_icu' ], axis=1, inplace=True)

In [ ]:
cv_patients.shape

(45155, 3)

In [ ]:
def missing(df):
        # Total missing values
        val = df.isnull().sum()
        
        # Percentage of missing values
        percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        table = pd.concat([val, percent], axis=1)
        
        # Rename the columns
        table_ren_columns = table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        table_ren_columns = table_ren_columns[table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
        # Return the dataframe with missing information
        return table_ren_columns

In [ ]:
#call missing function
missing(cv_patients)

,Missing Values,% of Total Values


No missing values

In [ ]:
IDs = list(set(cv_patients['icustay_id'].tolist())) # get all the time series ID's
num_samples = len(IDs)
print(num_samples)

45155


##Vital signs and lab tests

In [ ]:
df_vitals_labs.head()

,subject_id,hadm_id,icustay_id,hours_in,alanine aminotransferase_mean,alanine aminotransferase_std,albumin_mean,albumin_std,albumin ascites_mean,albumin ascites_std,...,weight_mean,weight_std,white blood cell count_mean,white blood cell count_std,white blood cell count urine_mean,white blood cell count urine_std,ph_mean,ph_std,ph urine_mean,ph urine_std
0,3,145834,211552,0,25.0,0.0,1.8,0.0,NaN,NaN,...,NaN,NaN,14.842857,4.012837,NaN,NaN,7.40,0.147733,5.0,NaN
1,3,145834,211552,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,145834,211552,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7.26,0.000000,NaN,NaN
3,3,145834,211552,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,145834,211552,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_vitals_labs.shape

(808539, 212)

In [ ]:
IDs = list(set(df_vitals_labs['icustay_id'].tolist())) # get all the time series ID's
num_samples = len(IDs)
print(num_samples)

34472


##Statistics

In [ ]:
df_stat.head()

,subject_id,hadm_id,icustay_id,gender,ethnicity,age,insurance,fullcode_first,dnr_first,fullcode,...,cmo_first,cmo_last,cmo,los_icu,admission_type,first_careunit,mort_icu,hospstay_seq,readmission_30,max_hours
0,3,145834,211552,M,WHITE,76,Medicare,1.0,0.0,1.0,...,0.0,0.0,0.0,6.064560,EMERGENCY,MICU,0,1,0,145
1,4,185777,294638,F,WHITE,47,Private,1.0,0.0,1.0,...,0.0,0.0,0.0,1.678472,EMERGENCY,MICU,0,1,0,40
2,6,107064,228232,F,WHITE,65,Medicare,1.0,0.0,1.0,...,0.0,0.0,0.0,3.672917,ELECTIVE,SICU,0,1,0,88
3,9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41,Medicaid,1.0,0.0,1.0,...,0.0,0.0,0.0,5.323056,EMERGENCY,MICU,1,1,0,127
4,11,194540,229441,F,WHITE,50,Private,1.0,0.0,1.0,...,0.0,0.0,0.0,1.584410,EMERGENCY,SICU,0,1,0,38


In [ ]:
df_stat.shape

(34472, 21)

In [ ]:
df_stat.shape

(500, 22)

In [ ]:
IDs = list(set(df_stat['icustay_id'].tolist())) # get all the time series ID's
num_samples = len(IDs)
print(num_samples)

34472


In [ ]:
IDs = list(set(df_stat['icustay_id'].tolist())) # get all the time series ID's
num_samples = len(IDs)
print(num_samples)

500


##Merge statistics with cardiovascular patients

In [ ]:
df_stat_cv  = pd.merge(df_stat,cv_patients,on='icustay_id'
                      , how= 'inner', suffixes=('', '_drop')).filter(regex='^(?!.*_drop)')

In [ ]:
df_stat_cv.head()

,index,subject_id,hadm_id,icustay_id,gender,ethnicity,age,insurance,fullcode_first,dnr_first,...,cmo_first,cmo_last,cmo,los_icu,admission_type,first_careunit,mort_icu,hospstay_seq,readmission_30,max_hours
0,9682,15993,121682,216439,F,WHITE,51,Private,1.0,0.0,...,0.0,0.0,0.0,1.520579,EMERGENCY,SICU,0,1,0,36
1,5930,9823,114757,220875,F,WHITE,77,Medicare,1.0,0.0,...,0.0,0.0,0.0,2.070822,EMERGENCY,SICU,0,1,0,49
2,20382,41055,122724,256046,F,UNKNOWN/NOT SPECIFIED,81,Medicare,1.0,0.0,...,0.0,0.0,0.0,4.322153,EMERGENCY,CCU,0,1,0,103
3,25906,63878,101540,214122,F,WHITE,50,Government,NaN,NaN,...,NaN,NaN,NaN,1.518356,ELECTIVE,SICU,0,1,0,36
4,1352,2136,174241,224238,M,BLACK/AFRICAN AMERICAN,64,Medicare,1.0,0.0,...,0.0,0.0,0.0,1.498993,EMERGENCY,CCU,0,1,0,35


In [ ]:
df_stat_cv.shape

(398, 21)

In [ ]:
IDs = list(set(df_stat_cv['icustay_id'].tolist())) # get all the time series ID's
num_samples = len(IDs)
print(num_samples)

28275


In [ ]:
IDs = list(set(df_stat_cv['subject_id'].tolist())) # get all the time series ID's
num_samples = len(IDs)
print(num_samples)

28275


##Merge the statistics of cardiovascular patients with vital signs and lab tests

In [ ]:
df_stat_cv_vitals_labs = pd.merge(df_stat_cv,df_vitals_labs,on='icustay_id'
                                  ,how='left', suffixes=('', '_drop')).filter(regex='^(?!.*_drop)')

In [ ]:
df_stat_cv_vitals_labs.head()

,subject_id,hadm_id,icustay_id,gender,ethnicity,age,insurance,fullcode_first,dnr_first,fullcode,...,weight_mean,weight_std,white blood cell count_mean,white blood cell count_std,white blood cell count urine_mean,white blood cell count urine_std,ph_mean,ph_std,ph urine_mean,ph urine_std
0,15993,121682,216439,F,WHITE,51,Private,1.0,0.0,1.0,...,NaN,NaN,11.4,NaN,NaN,NaN,NaN,NaN,5.0,NaN
1,15993,121682,216439,F,WHITE,51,Private,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15993,121682,216439,F,WHITE,51,Private,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15993,121682,216439,F,WHITE,51,Private,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15993,121682,216439,F,WHITE,51,Private,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_stat_cv_vitals_labs.shape

(9325, 230)

In [ ]:
IDs = list(set(df_stat_cv_vitals_labs['icustay_id'].tolist())) # get all the time series ID's
num_samples = len(IDs)
print(num_samples)

28275


In [ ]:
IDs = list(set(df_stat_cv_vitals_labs['subject_id'].tolist())) # get all the time series ID's
num_samples = len(IDs)
print(num_samples)

28275


Check ICU hours

some of them have less than 24 hours

In [ ]:
count_hours_icu_stat_cv_vitals_labs = df_stat_cv_vitals_labs.groupby('icustay_id').count()
count_hours_icu_stat_cv_vitals_labs.head()

,index,subject_id,hadm_id,gender,ethnicity,age,insurance,fullcode_first,dnr_first,fullcode,...,weight_mean,weight_std,white blood cell count_mean,white blood cell count_std,white blood cell count urine_mean,white blood cell count urine_std,ph_mean,ph_std,ph urine_mean,ph urine_std
icustay_id,,,,,,,,,,,,,,,,,,,,,
200338,24,24,24,24,24,24,24,24,24,24,...,0,0,2,2,0,0,2,2,0,0
200586,16,16,16,16,16,16,16,0,0,0,...,1,0,1,1,1,0,2,2,1,0
201652,24,24,24,24,24,24,24,24,24,24,...,0,0,3,3,0,0,3,3,1,0
201828,24,24,24,24,24,24,24,24,24,24,...,0,0,2,2,0,0,0,0,0,0
201838,24,24,24,24,24,24,24,24,24,24,...,1,0,2,1,0,0,1,1,1,0


In [ ]:
df_stat_cv_vitals_labs.to_csv('/content/drive/MyDrive/final_project/stat_cv_vitals_labs.csv')